In [ ]:
# drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Library

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# import timm

from tqdm.auto import tqdm
from copy import deepcopy

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
# import ttach as tta

import torchvision.models as models

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# device 할당
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# 권석원 ver
CFG = {
    'IMG_SIZE_H': 220,
    'IMG_SIZE_W': 275,
    'EPOCHS': 50,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 64,
    'SEED': 41,
    'PATIENCE' : 3
}

In [ ]:
# RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/train.csv')
train.shape

(5911, 3)

In [ ]:
train.loc[train['id'] == 3896,]

,id,img_path,artist
3896,3896,./train/3896.jpg,Edgar Degas
3986,3896,./train/3896.jpg,Titian


In [ ]:
train.loc[train.index == 3896, 'artist'] = 'Titian'
train.loc[train.index == 3986] = [3986, './train/3986.jpg', 'Alfred Sisley']


In [ ]:
train.loc[train.index == 3986]

,id,img_path,artist
3986,3986,./train/3986.jpg,Alfred Sisley


In [ ]:
train

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch
...,...,...,...
5906,5906,./train/5906.jpg,Pieter Bruegel
5907,5907,./train/5907.jpg,Peter Paul Rubens
5908,5908,./train/5908.jpg,Paul Gauguin
5909,5909,./train/5909.jpg,Paul Gauguin


In [ ]:
info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/artists_info.csv')
le = preprocessing.LabelEncoder()
info['genre'] = le.fit_transform(info['genre'])
info.rename(columns={'name':'artist'}, inplace=True)

In [ ]:
len(info['genre'].unique())

31

In [ ]:
new_train = pd.merge(train, info[['artist', 'genre']], on = 'artist', how='left')
new_train.isna().sum()

id            0
img_path      0
artist        0
genre       220
dtype: int64

In [ ]:
# 제대로 매칭이 안되었던 라벨이 현재 NaN으로 표시되므로 제대로 매칭 시켜주기
new_train = new_train.replace(np.nan, 14)

In [ ]:
def img_path_change(img_path):
  return '/content/drive/MyDrive/Colab Notebooks/Artist_classification/data' + str(img_path)[1:]

# train['img_path'] = train['img_path'].apply(img_path_change)
new_train['img_path'] = new_train['img_path'].apply(img_path_change)

## Train / Validation Split

In [ ]:
new_train['genre'] = new_train['genre'].astype(int)

In [ ]:
train_df, val_df, _, _ = train_test_split(new_train, new_train['genre'].values, test_size=0.2, random_state=CFG['SEED'])

In [ ]:
train_df = train_df.sort_values(by=['id'])
train_df.head()

,id,img_path,artist,genre
0,0,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Diego Velazquez,1
2,2,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Claude Monet,10
3,3,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Edgar Degas,10
5,5,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Pierre-Auguste Renoir,10
6,6,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Rene Magritte,26


In [ ]:
val_df = val_df.sort_values(by=['id'])
val_df.head()

,id,img_path,artist,genre
1,1,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Vincent van Gogh,16
4,4,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Hieronymus Bosch,14
17,17,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Edgar Degas,10
21,21,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Leonardo da Vinci,8
29,29,/content/drive/MyDrive/데이콘/월간 데이콘 ...,Kazimir Malevich,24


In [ ]:
train_df['img_path'][0]

In [ ]:
# inference=True면 test 데이터라는 뜻.
# 따라서 target에 해당하는 artist를 return할 수 없음.
def get_data(df, infer=False):
  if infer:
    return df['img_path'].values
    
  return df['img_path'].values, df['genre'].values

In [ ]:
# 파일 경로, 레이블
train_img_paths, train_labels = get_data(train_df)
val_img_paths, val_labels = get_data(val_df)

In [ ]:
# 여기서 9등분하고 train_imgs_labels, val_imgs_labels 만들기
def split_image(paths,labels):
  img_list = []
  label_list = []
  real_img_ls = []
  for path, label in tqdm(zip(paths, labels)):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    height,width,c = image.shape

    half_h = height//2
    half_w = width//2
    queter_h = half_h//2
    queter_w = half_w//2

    pos_list = [
        [0,half_w, 0,half_h],
        [half_w,width, 0,half_h],
        [0,half_w, half_h,height],
        [half_w,width, half_h,height],
        [0,half_w, queter_h,height-queter_h],
        [half_w,width, queter_h,height-queter_h],
        [queter_w,width-queter_w, 0,half_h],
        [queter_w,width-queter_w, half_h,height],
        [queter_w,width-queter_w, queter_h,height-queter_h]
    ]

    for poses in pos_list:
      img_list.append(image[poses[2]:poses[3],poses[0]:poses[1]])
      label_list.append(label)

  return img_list,label_list

In [ ]:
train_imgs, train_labels = split_image(train_img_paths, train_labels)

0it [00:00, ?it/s]

In [ ]:
val_imgs, val_labels = split_image(val_img_paths, val_labels)

0it [00:00, ?it/s]

## CustomDataset

In [ ]:
# torch.utils.data.Dataset이라는 class를 상속받는 자식 클래스
class CustomDataset(Dataset):

  # 데이터셋을 처음 선언할 때, 자동으로 호출.
  # 몇 가지 인수들을 입력받도록 만들 수 있다.
  def __init__(self, imgs, labels, transforms=None):
    self.imgs = imgs
    self.labels = labels
    self.transforms = transforms

  # 데이터셋에서 특정 1개의 샘플을 가져오기
  # index는 몇 번째 데이터를 가져올건지에 대한 변수.
  def __getitem__(self, index):
    image = self.imgs[index]

    # 아래 dataset 선언을 보면 transform이 사용됨.
    if self.transforms is not None:
      image = self.transforms(image=image)['image']

    if self.labels is not None:
      label = self.labels[index]
      return image, label
    else:
      return image

  # 데이터셋의 길이 (총 샘플의 수)
  # 데이터셋을 선언하고 dataloader를 사용할 때 내부적으로 사용
  ## 데이터셋의 len을 알아야 데이터로더가 미니 배치를 사용할 수 있기 때문
  def __len__(self):
    return len(self.imgs)

In [ ]:
class TestDataset(Dataset):
  def __init__(self, img_paths, labels, transforms=None):
    self.img_paths = img_paths
    self.labels = labels
    self.transforms = transforms

  def __getitem__(self, index):
    img_path = self.img_paths[index]

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    if self.transforms is not None:
      image = self.transforms(image=image)['image']

    if self.labels is not None:
      label = self.labels[index]
      return image, label
    else:
      return image

  def __len__(self):
    return len(self.img_paths)

In [ ]:
# Albumentation Augmentation
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE_H'],CFG['IMG_SIZE_W']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE_H'],CFG['IMG_SIZE_W']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
# Data Loader
train_dataset = CustomDataset(train_imgs, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=2)

val_dataset = CustomDataset(val_imgs, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=2)

In [ ]:
len(le.classes_)

31

In [ ]:
import torchvision.models as models
class Network_eff_genre(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(Network_eff_genre, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True) # b0 ~ b7
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):
  # 모델을 device에 할당
  model.to(device)

  # early stopping
  es_count = 0

  # Loss 정의
  criterion = nn.CrossEntropyLoss().to(device)

  # Scheduler에서 사용할 변수 선언
  best_score = 0
  best_model = None

  for epoch in range(1, CFG['EPOCHS'] + 1):
    # model을 train 모드로 전환
    model.train()

    # loss값을 넣을 리스트 생성
    train_loss = []

    # Epoch 진행
    for img, label in tqdm(iter(train_loader)):
      img, label = img.float().to(device), label.to(device)

      # 과거에 이용한 mini batch 내 이미지, 레이블을 바탕으로 계산된 Loss의 Gradient값이 optimizer에 할당되어 있는 것을 방지.
      optimizer.zero_grad()

      # pred값 
      model_pred = model(img)

      # 선언한 Loss에 pred값과 정답을 넣기 
      loss = criterion(model_pred, label)

      # backpropagation
      loss.backward()

      # optimizer
      optimizer.step()

      # loss값 추가
      train_loss.append(loss.item())

    # 최종 loss값 생성
    tr_loss = np.mean(train_loss)

    val_loss, val_score = validation(model, criterion, test_loader, device)

    print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')

    # Scheduler
    if scheduler is not None:
      scheduler.step()

    es_count += 1

    # val_score을 기준으로 best model 선정
    if best_score < val_score:
      best_model = model
      best_score = val_score
      es_count = 0

      # checkpoint
      best_acc_model = deepcopy(model.state_dict())
      print("model save!!" + 'genre.pt')
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/Artist_classification/model/genre.pt')
      
    if es_count > CFG['PATIENCE']:
      print('Early Stopping')
      break

  return best_model

In [ ]:
# 이번 대회에서는 F1 score를 사용
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):

  # 모델을 평가용으로 전환 (dropout 등의 규제가 들어가지 않게 조절)
  model.eval()
  
  model_preds = []
  true_labels = []

  val_loss = []

  # 평가 단계에서 Gradient를 통해 파라미터 값이 업데이트되는 현상을 방지
  with torch.no_grad():
    for img, label in tqdm(iter(test_loader)):
      img, label = img.float().to(device), label.to(device)
      model_pred = model(img)
      loss = criterion(model_pred, label)
      val_loss.append(loss.item())

      model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
      true_labels += label.detach().cpu().numpy().tolist()

  val_f1 = competition_metric(true_labels, model_preds)
  return np.mean(val_loss), val_f1

In [ ]:
# Run1

model_eff_genre = Network_eff_genre()
model_eff_genre.eval()
optimizer = torch.optim.Adam(params = model_eff_genre.parameters(), lr = CFG["LEARNING_RATE"])

# scheduler
lambda1 = lambda epoch: 0.65 ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

infer_model_eff_genre = train(model_eff_genre, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/665 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.81945] Val Loss : [0.92666] Val F1 Score : [0.69323]
model save!!best_model_effv2_0.69.pt


  0%|          | 0/665 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

...

  0%|          | 0/167 [00:00<?, ?it/s]

Epoch [11], Train Loss : [0.01586] Val Loss : [0.90751] Val F1 Score : [0.77364]


  0%|          | 0/665 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

Epoch [12], Train Loss : [0.01461] Val Loss : [0.91122] Val F1 Score : [0.77354]
Early Stopping
